In [1]:
from langgraph.graph import StateGraph, add_messages, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


class State(TypedDict):
    text: str


def node_a(state: State):
    print("Node A")
    return Command(goto="node_b", update={"text": state["text"] + "a"})


def node_b(state: State):
    print("Node B")

    human_response = interrupt("Do you want to go to C or D? Type C/D")

    if human_response.upper() == "C":
        return Command(goto="node_c", update={"text": state["text"] + "b"})
    elif human_response.upper() == "D":
        return Command(goto="node_d", update={"text": state["text"] + "b"})


def node_c(state: State):
    print("Node C")
    return Command(goto=END, update={"text": state["text"] + "c"})


def node_d(state: State):
    print("Node D")
    return Command(goto=END, update={"text": state["text"] + "d"})


print("Starting the graph")

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)
config = {
    "configurable": {
        "thread_id": 1,
    }
}

initial_state = {"text": ""}

response = app.invoke(
    initial_state,
    config=config,
    stream_mode="updates",
)

print(response)


Starting the graph
Node A
Node B
[{'node_a': {'text': 'a'}}, {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:12be7900-eba7-0628-55a8-8cf45c102de5']),)}]


In [2]:
print(app.get_state(config).next)

('node_b',)


In [3]:
second_result = app.invoke(
    Command(resume="D"),
    config=config,
    stream_mode="updates",
)
second_result

Node B
Node D


[{'node_b': {'text': 'ab'}}, {'node_d': {'text': 'abd'}}]